# EP01: 대규모 쿠버네티스 클러스터 만들기

## 애저 로그인 상태 확인

우선 애저 커맨드 라인 도구의 로그인 상태를 먼저 확인합니다. 로그인 상태가 아니면 다른 터미널에서 `az login` 명령을 실행합니다. 설치되어있지 않다면 OS 버전 별로 애저 커맨드 라인 도구를 설치하고 준비합니다.

In [1]:
az account list

The below script needs to be able to find the current output cell; this is an easy method to get it.

Name               CloudName    SubscriptionId                        State    IsDefault
-----------------  -----------  ------------------------------------  -------  -----------
rkttu@outlook.com  AzureCloud   e6535fda-2c47-411d-9e51-ae11ef7bde27  Enabled  True


## AKS 엔진 설치 상태 확인

아래 명령어를 실행하여 `aks-engine`이 설치되었는지 확인합니다. 설치되어있지 않다면 OS 버전 별로 각각 설치 가이드를 따라 설치를 진행합니다.

In [2]:
aks-engine version

Version: v0.56.0
GitCommit: 666073d49
GitTreeState: clean


## 서비스 주체 생성

쿠버네티스 클러스터를 만든 후에는 클러스터 수준에서 스스로 애저 리소스들을 제어할 수 있어야 하므로, 서비스 주체 계정을 만들어 계속 해당 계정을 사용하도록 설정합니다.

In [4]:
$SubscriptionData = (az account list -o json) | ConvertFrom-Json -Depth 16
$SubscriptionId = $SubscriptionData[0].id
$SvcPrincipal = (az ad sp create-for-rbac `
  -n="AzurePlaylist" --role="Contributor" `
  --scopes="/subscriptions/$SubscriptionId" `
  -o json) | ConvertFrom-Json -Depth 16

$PrincipalId = $SvcPrincipal.AppId
$PrincipalPwd = $SvcPrincipal.Password

"Your App ID is $PrincipalId"

Changing "AzurePlaylist" to a valid URI of "http://AzurePlaylist", which is the required format used for service principal names
Found an existing application instance of "5f6ed45a-660e-47f5-9c08-6e5515f17640". We will patch it
Creating a role assignment under the scope of "/subscriptions/e6535fda-2c47-411d-9e51-ae11ef7bde27"
  Role assignment already exists.

Your App ID is 5f6ed45a-660e-47f5-9c08-6e5515f17640


## SSH 키 준비하기

아래 명령어를 실행하여 SSH 키의 공개 키를 준비합니다. 설치되어있지 않다면 SSH 키 쌍을 만듭니다.

In [5]:
$PublicKeyValue = Get-Content -Path "$HOME\.ssh\id_rsa.pub"

## API 모델 준비하기

이제 API 모델을 준비할 차례입니다. AKS 엔진은 코드 기반 인프라 관리 방식을 구현하기 위하여 JSON API 모델을 작성하면, 여기에 맞추어 ARM 템플릿을 만들어줍니다. 이것을 애저 명령줄 도구를 통해 배포하면 손쉽게 쿠버네티스 클러스터를 생성할 수 있습니다.

In [6]:
$ApiModel = @"
{
  'apiVersion': 'vlabs',
  'properties': {
    'orchestratorProfile': {
      'orchestratorType': 'Kubernetes',
      'orchestratorRelease': '1.16',
      'kubernetesConfig': {
        'cloudProviderBackoff': true,
        'cloudProviderBackoffRetries': 6,
        'cloudProviderBackoffJitter': 1,
        'cloudProviderBackoffDuration': 6,
        'cloudProviderBackoffExponent': 1.5,
        'cloudProviderRateLimit': true,
        'cloudProviderRateLimitQPS': 3,
        'cloudProviderRateLimitBucket': 10,
        'kubeletConfig': {
          '--node-status-update-frequency': '1m'
        },
        'controllerManagerConfig': {
          '--node-monitor-grace-period': '5m',
          '--pod-eviction-timeout': '1m',
          '--route-reconciliation-period': '1m'
        }
      }
    },
    'servicePrincipalProfile': {
      'clientId': '',
      'secret': ''
    },
    'masterProfile': {
      'count': 1,
      'vmSize': 'Standard_D5_v2'
    },
    'agentPoolProfiles': [
      {
        'name': 'pool1',
        'count': 5,
        'vmSize': 'Standard_D3_v2',
        'availabilityProfile': 'AvailabilitySet'
      },
      {
        'name': 'pool2',
        'count': 5,
        'vmSize': 'Standard_D3_v2',
        'availabilityProfile': 'AvailabilitySet'
      },
      {
        'name': 'pool3',
        'count': 5,
        'vmSize': 'Standard_D3_v2',
        'availabilityProfile': 'AvailabilitySet'
      },
      {
        'name': 'pool4',
        'count': 5,
        'vmSize': 'Standard_D3_v2',
        'availabilityProfile': 'AvailabilitySet'
      },
      {
        'name': 'pool5',
        'count': 5,
        'vmSize': 'Standard_D3_v2',
        'availabilityProfile': 'AvailabilitySet'
      }
    ],
    'linuxProfile': {
      'adminUsername': 'azureuser',
      'ssh': {
        'publicKeys': [
          {
            'keyData': ''
          }
        ]
      }
    }
  }
}
"@ | ConvertFrom-Json -Depth 16

## API 모델 내용 변경하기

나중에 리눅스 마스터 노드에 접속할 수 있도록 현재 컴퓨터에서 사용하는 SSH 공개 키를 배포할 때 같이 등록합니다. 그리고 서비스 주체를 가리키는 아이디 값과 고유 비밀 키 값을 API 모델에 같이 지정합니다.

In [7]:
$ApiModel.properties.linuxProfile.ssh.publicKeys[0].keyData = "$PublicKeyValue"
$ApiModel.properties.servicePrincipalProfile.clientId = "$PrincipalId"
$ApiModel.properties.servicePrincipalProfile.secret = "$PrincipalPwd"

In [8]:
$ApiModel | ConvertTo-Json -Depth 16 | Out-File -Encoding utf8 -Path 'ep01.json' -Force

dir 'ep01*'



    Directory: /Users/rkttu/Projects/azure-playlist-aks-engine

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d----      2020. 11. 8.  오후 6:41                ep01_output
d----      2020. 11. 8.  오후 4:14                translations
-----      2020. 11. 8.  오후 6:49          17264 EP01.ipynb
-----      2020. 11. 8.  오후 6:50           2395 ep01.json



## API 모델을 ARM 템플릿으로 변환하고 배포하기

다음의 명령어를 실행하여 API 모델을 ARM 템플릿으로 변환하고 곧바로 애저에 배포하도록 하겠습니다. 이 명령어는 최초에 한 번만 실행하고, 그 이후에는 `ep01_output` 디렉터리 안에 있는 파일들을 이용하여 클러스터를 관리하도록 합니다.

In [87]:
$DnsPrefix = 'rkttuep01'
$ResourceGroup = 'rkttuep01'
$Location = 'westus2'

aks-engine deploy --dns-prefix "$DnsPrefix" `
    --resource-group "$ResourceGroup" `
    --location "$Location" `
    --api-model "ep01.json" `
    --auto-suffix `
    --output-directory "ep01_output" `
    --auth-method cli

INFO[0000] No subscription provided, using selected subscription from azure CLI: e6535fda-2c47-411d-9e51-ae11ef7bde27 
WARN[0003] apimodel: missing masterProfile.dnsPrefix will use "rkttuep01" 
WARN[0004] containerd will be upgraded to version 1.3.7 
INFO[0008] Starting ARM Deployment rkttuep01-1800061721 in resource group rkttuep01. This will take some time... 
INFO[0243] Finished ARM Deployment (rkttuep01-1800061721). Succeeded 


## kubectl 실행해보기

클러스터가 잘 만들어졌는지 확인해보기 위하여 다음과 같이 `KUBECONFIG` 환경 변수를 설정하여 클러스터에 접근하고, 노드와 구성 상태를 확인합니다.

In [88]:
$env:KUBECONFIG="ep01_output/kubeconfig/kubeconfig.$Location.json"
kubectl get nodes -o wide

NAME                    STATUS   ROLES    AGE   VERSION    INTERNAL-IP    EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION     CONTAINER-RUNTIME
k8s-master-22285348-0   Ready    master   77s   v1.16.15   10.255.255.5   <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-pool1-22285348-0    Ready    agent    77s   v1.16.15   10.240.0.4     <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-pool1-22285348-1    Ready    agent    72s   v1.16.15   10.240.2.80    <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-pool1-22285348-2    Ready    agent    74s   v1.16.15   10.240.0.220   <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-pool1-22285348-3    Ready    agent    77s   v1.16.15   10.240.0.189   <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-pool1-22285348-4    Ready    agent    77s   v1.16.15   10.240.0.35    <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   d

## 리소스 그룹 정리

테스트를 위해 생성한 리소스 그룹을 제거하고 모든 리소스를 일괄 정리합니다.

In [89]:
az group delete --name $ResourceGroup --yes --no-wait

## 감사합니다!